# Predicting the popularity of tweets

In [281]:
# Import all libraries needed through the whole notebook
import pandas as pd

# Exploratory data analysis

During the exploratory data analysis, we will look at the data to understand its structure and the relationships between the variables. We have two main goals during this phase:
1. Choose which variable will be our target variable
2. Identify the most important variables that will help us predict the target variable

## Load the data

The first step is to load the data and check the information about the columns and the data types are correct. One special thing to consider the data is that we have two datasets: one with the tweets and another with the users. We will need to join these datasets to have all the information in one place.

In [282]:
# Load both datasets
tweets = pd.read_csv('./data/tweets.csv', low_memory=False)
users = pd.read_csv('./data/users.csv')

In [283]:
# Join the datasets
users.set_index('id', inplace=True)
tweets_users = tweets.join(users, on='user_id', lsuffix='_tweet', rsuffix='_user', how='left')

### Check data has been loaded correctly

In [284]:
tweets_users

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,tweets,location,following,followers,likes,media,private,verified,avatar,background_image
0,1425590913959612419,1425590913959612419,1.628722e+12,2021-08-12 00:52:14,200,NaN,RT @girlsalliance: We're so proud of the four ...,en,[],[],...,1770,"Washington, DC",16,20854298,184,461,False,True,https://pbs.twimg.com/profile_images/136674780...,https://pbs.twimg.com/profile_banners/40948655...
1,1427736867739299841,1427736867739299841,1.629234e+12,2021-08-17 22:59:29,200,NaN,Some casual suggestions to 😏SLIDE😏 into when u...,en,"['shoesdaytuesday', 'afterskewlslide']",[],...,11420,NaN,235,108819032,7995,2170,False,True,https://pbs.twimg.com/profile_images/139246535...,https://pbs.twimg.com/profile_banners/21447363...
2,1427667300488937476,1427667300488937476,1.629217e+12,2021-08-17 18:23:03,200,NaN,RT @ValaAfshar: You are not your job.,en,[],[],...,11420,NaN,235,108819032,7995,2170,False,True,https://pbs.twimg.com/profile_images/139246535...,https://pbs.twimg.com/profile_banners/21447363...
3,1427667012105371652,1427667012105371652,1.629217e+12,2021-08-17 18:21:55,200,NaN,What have we become 😔😂 Toddler Cites Freedom ...,en,[],[],...,11420,NaN,235,108819032,7995,2170,False,True,https://pbs.twimg.com/profile_images/139246535...,https://pbs.twimg.com/profile_banners/21447363...
4,1427497703596990467,1427497703596990467,1.629177e+12,2021-08-17 07:09:08,200,NaN,The tech giants that refuse to massively addre...,en,[],[],...,11420,NaN,235,108819032,7995,2170,False,True,https://pbs.twimg.com/profile_images/139246535...,https://pbs.twimg.com/profile_banners/21447363...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40576,1427593440469061634,1427593440469061634,1.629200e+12,2021-08-17 13:29:34,200,NaN,Commencement of works. #Agenda111 https://t.c...,en,['agenda111'],[],...,7744,Ghana,352,2003463,338,1499,False,True,https://pbs.twimg.com/profile_images/817691975...,https://pbs.twimg.com/profile_banners/24721710...
40577,1427592955272089642,1427592930722820096,1.629200e+12,2021-08-17 13:27:38,200,NaN,Commencement of works. #Agenda111 https://t.c...,en,['agenda111'],[],...,7744,Ghana,352,2003463,338,1499,False,True,https://pbs.twimg.com/profile_images/817691975...,https://pbs.twimg.com/profile_banners/24721710...
40578,1427592942441598980,1427592930722820096,1.629200e+12,2021-08-17 13:27:35,200,NaN,Commencement of works. #Agenda111 https://t.c...,en,['agenda111'],[],...,7744,Ghana,352,2003463,338,1499,False,True,https://pbs.twimg.com/profile_images/817691975...,https://pbs.twimg.com/profile_banners/24721710...
40579,1427592930722820096,1427592930722820096,1.629200e+12,2021-08-17 13:27:32,200,NaN,Commencement of works. #Agenda111 https://t.c...,en,['agenda111'],[],...,7744,Ghana,352,2003463,338,1499,False,True,https://pbs.twimg.com/profile_images/817691975...,https://pbs.twimg.com/profile_banners/24721710...


At first glance, we see that some columns do not have the expected data types. We will convert them to the correct data types before we make any analysis to make sure we don't have any problems with the data types.

In [285]:
# Convert columns to the correct data types
tweets_users['id'] = tweets_users['id'].astype('str')
tweets_users['conversation_id'] = tweets_users['conversation_id'].astype('str')
tweets_users['created_at'] = pd.to_datetime(tweets_users['created_at'], unit='ms')
tweets_users['date'] = pd.to_datetime(tweets_users['date'])
tweets_users['user_id'] = tweets_users['user_id'].astype('str')
tweets_users['user_id_str'] = tweets_users['user_id_str'].astype('str')
tweets_users['video'] = tweets_users['video'].astype('bool')
tweets_users['user_rt_id'] = tweets_users['user_rt_id'].astype('str')
tweets_users['retweet_id'] = tweets_users['retweet_id'].astype('str')
tweets_users['retweet_date'] = pd.to_datetime(tweets_users['retweet_date'].str.rstrip(' CEST'))
tweets_users['join_datetime'] = pd.to_datetime(tweets_users['join_datetime'])
tweets_users['join_date'] = pd.to_datetime(tweets_users['join_date'])
tweets_users['join_time'] = pd.to_datetime(tweets_users['join_time'], format='%H:%M:%S %Z').dt.time
tweets_users['private'] = tweets_users['private'].astype('bool')
tweets_users['verified'] = tweets_users['verified'].astype('bool')

In [286]:
tweets_users.shape

(40867, 55)

In [287]:
tweets_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40867 entries, 0 to 40580
Data columns (total 55 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                40867 non-null  object             
 1   conversation_id   40867 non-null  object             
 2   created_at        40867 non-null  datetime64[ns]     
 3   date              40867 non-null  datetime64[ns]     
 4   timezone          40867 non-null  int64              
 5   place             34 non-null     object             
 6   tweet             40867 non-null  object             
 7   language          40867 non-null  object             
 8   hashtags          40867 non-null  object             
 9   cashtags          40867 non-null  object             
 10  user_id           40867 non-null  object             
 11  user_id_str       40867 non-null  object             
 12  username_tweet    40867 non-null  object             
 13  name_t

In [288]:
tweets_users.describe()

,created_at,date,timezone,day,hour,nlikes,nreplies,nretweets,search,near,...,retweet_date,translate,trans_src,trans_dest,join_date,tweets,following,followers,likes,media
count,40867,40867,40867.0,40867.000000,40867.000000,4.086700e+04,40867.000000,40867.000000,0.0,0.0,...,6182,0.0,0.0,0.0,40867,40867.000000,4.086700e+04,4.086700e+04,40867.000000,40867.000000
mean,2021-08-15 13:00:53.727041280,2021-08-15 15:00:53.727041280,200.0,3.816013,12.999560,1.417170e+03,78.194118,288.468569,NaN,NaN,...,2021-08-12 23:04:28.001941248,NaN,NaN,NaN,2009-01-17 07:22:25.584456960,177398.772971,1.152955e+04,1.010760e+07,13963.126728,50492.908190
min,2021-08-11 22:00:00,2021-08-12 00:00:00,200.0,1.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,NaN,...,2015-09-01 12:10:02,NaN,NaN,NaN,2006-03-21 00:00:00,5.000000,0.000000e+00,1.070000e+02,0.000000,5.000000
25%,2021-08-13 14:56:50,2021-08-13 16:56:50,200.0,2.000000,6.000000,6.000000e+00,0.000000,4.000000,NaN,NaN,...,2021-08-13 05:30:01.249999872,NaN,NaN,NaN,2007-05-12 00:00:00,40113.000000,4.180000e+02,3.039465e+06,353.000000,3402.000000
50%,2021-08-15 15:30:01,2021-08-15 17:30:01,200.0,4.000000,15.000000,3.900000e+01,3.000000,16.000000,NaN,NaN,...,2021-08-15 00:38:46,NaN,NaN,NaN,2008-10-09 00:00:00,132248.000000,8.620000e+02,4.878881e+06,2600.000000,22586.000000
75%,2021-08-17 13:45:32,2021-08-17 15:45:32,200.0,5.000000,19.000000,2.060000e+02,24.000000,74.000000,NaN,NaN,...,2021-08-16 23:37:16,NaN,NaN,NaN,2009-10-22 00:00:00,306127.000000,2.265000e+03,1.025547e+07,8759.000000,75519.000000
max,2021-08-18 22:00:00,2021-08-19 00:00:00,200.0,7.000000,23.000000,1.920242e+06,88035.000000,541964.000000,NaN,NaN,...,2021-08-18 23:51:40,NaN,NaN,NaN,2021-06-21 00:00:00,508811.000000,4.200793e+06,1.299093e+08,492144.000000,236024.000000
std,NaN,NaN,0.0,1.887138,7.328175,1.824648e+04,983.854499,4644.698916,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,148999.823601,1.628862e+05,1.322146e+07,51451.295821,62376.057012


In [289]:
tweets_users.nunique()

id                  40581
conversation_id     37273
created_at          36756
date                36756
timezone                1
place                  22
tweet               40026
language               40
hashtags             2561
cashtags                4
user_id               847
user_id_str           847
username_tweet        847
name_tweet            847
day                     7
hour                   24
link                40581
urls                19576
photos               5344
video                   2
thumbnail            8661
retweet                 2
nlikes               4573
nreplies             1258
nretweets            2281
quote_url            2446
search                  0
near                    0
geo                     0
source                  0
user_rt_id           3560
user_rt              6046
retweet_id           6048
reply_to             3576
retweet_date         5972
translate               0
trans_src               0
trans_dest              0
name_user   

In [290]:
tweets_users.isnull().sum()

id                      0
conversation_id         0
created_at              0
date                    0
timezone                0
place               40833
tweet                   0
language                0
hashtags                0
cashtags                0
user_id                 0
user_id_str             0
username_tweet          0
name_tweet              0
day                     0
hour                    0
link                    0
urls                    0
photos                  0
video                   0
thumbnail           31644
retweet                 0
nlikes                  0
nreplies                0
nretweets               0
quote_url               0
search              40867
near                40867
geo                 40867
source              40867
user_rt_id              0
user_rt             34685
retweet_id              0
reply_to                0
retweet_date        34685
translate           40867
trans_src           40867
trans_dest          40867
name_user   

The dataset has been loaded correctly, no issues were found during the process, and the data is coherent with what we could expect from each of the variables.

